In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from scipy.spatial import Voronoi
from sklearn.model_selection import train_test_split

In [4]:
#Useful functions

def create_ffn(hidden_units, dropout_rate, input_shape=None, name=None):
    
    #Creates a sequential model (feed-forward network) 
   
    fnn_layers = []
    if input_shape is not None:
        fnn_layers.append(layers.Input(shape=input_shape))
    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))
    return keras.Sequential(fnn_layers, name=name)

def create_gru(hidden_units, dropout_rate):
    
    #Creates a GRU based model for combining nodes information
    
    inputs = keras.layers.Input(shape=(2, hidden_units[0]))
    x = inputs
    for units in hidden_units:
        x = layers.GRU(
            units=units,
            activation="tanh",
            recurrent_activation="sigmoid",
            return_sequences=True,
            dropout=dropout_rate,
            recurrent_dropout=dropout_rate
        )(x)
    return keras.Model(inputs=inputs, outputs=x)

#Convolution layer

class GraphConvLayer(layers.Layer):
    def __init__(self, hidden_units, dropout_rate=0.2, aggregation_type="mean",
                 combination_type="concat", normalize=False, *args, **kwargs):

        # Layer that processes messages in a graph: prepares messages from neighbours with a FFN, 
        # aggregates messages of neighbours through a specified method (sum,mean,max) and combines 
        # the node representation with the aggregated message

        super(GraphConvLayer, self).__init__(*args, **kwargs)
        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize
        self.hidden_units = hidden_units
        self.dropout_rate = dropout_rate

        # FFN para preparar mensajes
        self.ffn_prepare = create_ffn(hidden_units, dropout_rate, name="ffn_prepare")
        # Función de actualización: puede ser una GRU o una FFN
        if self.combination_type == "gru":
            self.update_fn = create_gru(hidden_units, dropout_rate)
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate, name="update_ffn")

    def build(self, input_shape):
        #We can implement the variable initialization here if necessary 
        super(GraphConvLayer, self).build(input_shape)

    def prepare(self, node_representations, weights=None):
        messages = self.ffn_prepare(node_representations)
        if weights is not None:
            messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_representations):
        # As it can vary between images, we use the number of nodes dinamically
        # node_indices shape is [num_edges].
        # neighbour_messages shape: [num_edges, representation_dim].
        # node_repesentations shape is [num_nodes, representation_dim]
        num_nodes = tf.shape(node_representations)[0]
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(neighbour_messages, node_indices, num_segments=num_nodes)
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(neighbour_messages, node_indices, num_segments=num_nodes)
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(neighbour_messages, node_indices, num_segments=num_nodes)
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")
        return aggregated_message

    def update(self, node_representations, aggregated_messages):
        # node_repesentations shape is [num_nodes, representation_dim].
        # aggregated_messages shape is [num_nodes, representation_dim].
        if self.combination_type == "gru":
            h = tf.stack([node_representations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            h = tf.concat([node_representations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            h = node_representations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")
        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            # Seleccionamos la salida final de la secuencia GRU
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]
        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):
        """Process the inputs to produce the node_embeddings.

        inputs: a tuple of three elements: node_repesentations, edges, edge_weights.
            -node_representations: tensor with shape (num_nodes,feature_dim)
            -edges: tensor with shape (num_edges,2) 
            -edge_weights:with shape (num_edges,), as in our problem all the edges
            have the same weight this tensor is going to be a ones array
        Returns: node_embeddings of shape [num_nodes, representation_dim].
        """
        node_representations, edges, edge_weights = inputs
        # Divide the source and target indices
        source_indexes = edges[:, 0]
        target_indexes = edges[:, 1]
        # Obtain the neighbour (target) representations
        neighbour_representations = tf.gather(node_representations, target_indexes)
        neighbour_messages = self.prepare(neighbour_representations, edge_weights)
        aggregated_messages = self.aggregate(source_indexes, neighbour_messages, node_representations)
        return self.update(node_representations, aggregated_messages)

In [5]:
#Node Classifier model 

class GNNNodeClassifier(tf.keras.Model):
    def __init__(self, num_classes, hidden_units, aggregation_type="mean",
                 combination_type="concat", dropout_rate=0.2, normalize=True, *args, **kwargs):
        super(GNNNodeClassifier, self).__init__(*args, **kwargs)
        # Preprocessing: transform the node features
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Convolutional layers
        self.conv1 = GraphConvLayer(hidden_units, dropout_rate, aggregation_type,
                                    combination_type, normalize, name="graph_conv1")
        self.conv2 = GraphConvLayer(hidden_units, dropout_rate, aggregation_type,
                                    combination_type, normalize, name="graph_conv2")
        # Postprocessing
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Final layer that produces the logits for each node
        self.compute_logits = layers.Dense(units=num_classes, name="logits")

    def call(self, inputs):
        """
        Inputs should be a tuple of:
        (node_features, edges, edge_weights, input_node_indices)
        where:
            - node_features: tensor with shape (batch_size, num_nodes, feature_dim)
            - edges: tensor with shape (batch_size, num_edges, 2)
            - edge_weights: tensor with shape (batch_size, num_edges)
            - node_indices: tensor with shape (batch_size, num_nodes)
        Each input corresponds to a graph/image

        """
        node_features, edges, edge_weights, node_indices = inputs

        # Function that processes a single graph
        def process_graph(single_inputs):
            nf, e, ew, ni = single_inputs  # nf: (num_nodes, feature_dim), e: (num_edges, 2), etc.
            x = self.preprocess(nf)  # x: (num_nodes, hidden_dim)
            x1 = self.conv1((x, e, ew))
            x = x + x1  
            x2 = self.conv2((x, e, ew))
            x = x + x2  
            x = self.postprocess(x)
            # Obtain the representations for each node
            node_emb = tf.gather(x, ni)
            logits = self.compute_logits(node_emb)  # (num_nodes, num_classes)
            return logits

        # Apply tf.map_fn for processing each graph of the batch
        outputs = tf.map_fn(process_graph, (node_features, edges, edge_weights, node_indices),
                            fn_output_signature=tf.float32)
        # outputs have shape:(batch_size, num_nodes, num_classes)
        return outputs


In [6]:
#Functions for extracting data from the dataframe and building the dataset
def extract_graph_data(df, image_id):
    """
    Extracts data of the graph for a given image
      - Filters the rows with image_id.
      - Uses columns 'x' and 'y' por Voronoi tessellation
      - Extracts features for each node 
      - Label is the column 'activity'
    """
    df_img = df[df['image_id'] == image_id].reset_index(drop=True)
    num_nodes = df_img.shape[0]
    
    points = df_img[['x', 'y']].to_numpy()
    vor = Voronoi(points)
    # Obtains the edges (a pair of points) for the Voronoi tessellation
    if len(vor.ridge_points) > 0:
        edges = np.array(vor.ridge_points, dtype=np.int32)
    else:
        edges = np.empty((0, 2), dtype=np.int32)
    num_edges = edges.shape[0]
   
    edge_weights = np.ones((num_edges,), dtype=np.float32)
       
    feature_cols = [col for col in df_img.columns if col not in ['image_id', 'x', 'y', 'activity','label','type']]
    node_features = df_img[feature_cols].to_numpy().astype(np.float32)

    labels = df_img['activity'].to_numpy().astype(np.int32)
    # Modes indexes: just from 0 to num_nodes-1
    node_indexes = np.arange(num_nodes, dtype=np.int32)
    
    return node_features, edges, edge_weights, node_indexes, labels

#Creates a tf.data.Dataset from a dataframe
def create_graph_dataset(df, batch_size, feature_dim):
    image_ids = df['image_id'].unique()
    
    def gen():
        for img_id in image_ids:
            node_features, edges, edge_weights, node_indices, labels = extract_graph_data(df, img_id)
            # Ensure that the shapes are correct:
            node_features = np.reshape(node_features, (-1, feature_dim))
            # edges with shape:(num_edges, 2)
            edges = np.reshape(edges, (-1, 2))
            edge_weights = np.reshape(edge_weights, (-1,))
            node_indices = np.reshape(node_indices, (-1,))
            labels = np.reshape(labels, (-1,))
            yield (node_features, edges, edge_weights, node_indices), labels
    
    dataset = tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            (
                tf.TensorSpec(shape=(None, feature_dim), dtype=tf.float32),  # node_features
                tf.TensorSpec(shape=(None, 2), dtype=tf.int32),              # edges
                tf.TensorSpec(shape=(None,), dtype=tf.float32),              # edge_weights
                tf.TensorSpec(shape=(None,), dtype=tf.int32),                # node_indices
            ),
            tf.TensorSpec(shape=(None,), dtype=tf.int32)  # labels
        )
    )
    # Use padded_batch para handling graphs with a varying number of edges
    dataset = dataset.padded_batch(
        batch_size,
        padded_shapes=(
            (
                tf.TensorShape([None, feature_dim]),  # node_features
                tf.TensorShape([None, 2]),              # edges
                tf.TensorShape([None]),                # edge_weights
                tf.TensorShape([None]),                # node_indices
            ),
            tf.TensorShape([None])  # labels
        ),
        padding_values=(
            (
                tf.constant(0, dtype=tf.float32),
                tf.constant(0, dtype=tf.int32),
                tf.constant(0, dtype=tf.float32),
                tf.constant(0, dtype=tf.int32),
            ),
            tf.constant(-1, dtype=tf.int32)
        )
    )
    return dataset

In [16]:
#Load the input data
density=0.2
fa=100
input_file=f'phia{density}/traj_phia{density}-T05-Fa{fa}-tau1.dat'
df=pd.read_csv(input_file, sep='\s+',names=["label", "type", "x", "y"])
cols_names=['area', 'perimeter', 'neighbours', 'max neighbour distance',
       'min neighbour distance', 'max vertices distance',
       'min vertices distance', 'max vertices-point distance',
       'min vertices-point distance', 'distance to center', 'activity',
       'particle type']
input_file2=f"phia{density}/particles-features-{density}-Fa{fa}.txt"
df2=pd.read_csv(input_file2, sep='\s+',names=cols_names)

#Create a dataframe that includes both, the voronoi features and the particle positions
df=df[0:2_000_000].join(df2)
df['image_id']=np.floor(df.index/1000) #Add a column with the id of each image

In [17]:
import keras.metrics as metrics

In [18]:
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score
feature_cols = [col for col in df.columns if col not in ['image_id', 'x', 'y', 'activity','label','type']]
feature_dim = len(feature_cols)

# Model parameters
num_classes = 2       
hidden_units = [64, 64]
dropout_rate = 0.2
aggregation_type = "mean"
combination_type = "concat"
normalize = True
batch_size = 1  

images_ids=df['image_id'].unique()
train_images_ids,test_images_ids=train_test_split(images_ids,random_state=50,test_size=0.2)
train_df=df[df['image_id'].isin(train_images_ids)].reset_index(drop=True)
test_df=df[df['image_id'].isin(test_images_ids)].reset_index(drop=True)

# Create training and testing datasets
train_dataset = create_graph_dataset(train_df, batch_size, feature_dim)
test_dataset = create_graph_dataset(test_df, batch_size, feature_dim)

# Instance and compile the GNN model
model = GNNNodeClassifier(num_classes, hidden_units, aggregation_type, combination_type, dropout_rate, normalize)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [19]:
# Fit the model
model.fit(train_dataset, epochs=10)

Epoch 1/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.7887 - loss: 0.5027
Epoch 2/10
  8/320 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8055 - loss: 0.4654

c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


320/320 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8095 - loss: 0.4590
Epoch 3/10
  9/320 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8116 - loss: 0.4464

c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


320/320 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8142 - loss: 0.4474
Epoch 4/10
  9/320 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8156 - loss: 0.4419

...

c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


320/320 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.8215 - loss: 0.4246
Epoch 10/10
  8/320 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8228 - loss: 0.4210

c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


320/320 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8223 - loss: 0.4231


c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [20]:
#Evaluate the model
accuracy=model.evaluate(test_dataset)
print(accuracy[1])


81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8242 - loss: 0.4178
0.8239752650260925


c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [21]:
from sklearn.metrics import roc_auc_score

all_labels = []
all_predictions = []

for (node_features, edges, edge_weights, node_indices), labels in test_dataset:
    predictions = model((node_features, edges, edge_weights, node_indices))
    # probabilities = tf.nn.sigmoid(predictions).numpy()  # For binary classification
    probabilities = keras.activations.softmax(tf.convert_to_tensor(predictions)).numpy()
    decision=np.floor(probabilities[:,:,1]*2)
    all_labels.extend(labels.numpy().flatten())
    all_predictions.extend(decision.flatten())

In [22]:
from sklearn.metrics import f1_score,classification_report
auc = roc_auc_score(all_labels, all_predictions)
f1= f1_score(all_labels,all_predictions)
print("Test AUC:", auc)
print('Test F1:',f1)
print(classification_report(all_labels,all_predictions))

Test AUC: 0.5899382716049383
Test F1: 0.3123372238834764
              precision    recall  f1-score   support

           0       0.83      0.98      0.90     64800
           1       0.71      0.20      0.31     16200

    accuracy                           0.82     81000
   macro avg       0.77      0.59      0.61     81000
weighted avg       0.81      0.82      0.78     81000



In [88]:
all_predictions

[0.7159844,
 0.2645995,
 0.7345969,
 0.28207645,
 0.6380576,
 0.3489702,
 0.7453385,
 0.29966557,
 0.33867678,
 0.6462641,
 0.46524718,
 0.48047706,
 0.77662885,
 0.27569345,
 0.6975681,
 0.26362994,
 0.70750976,
 0.2806128,
 0.7573876,
 0.2490431,
 0.6658109,
 0.32538983,
 0.35746148,
 0.6140405,
 0.747182,
 0.24801345,
 0.6074776,
 0.35634506,
 0.7883099,
 0.20904529,
 0.6121916,
 0.34460166,
 0.7900176,
 0.23764397,
 0.31713736,
 0.62390727,
 0.79013336,
 0.24650258,
 0.4382181,
 0.5394414,
 0.6088959,
 0.36731318,
 0.5242623,
 0.4402898,
 0.8564378,
 0.17809601,
 0.42063662,
 0.56673616,
 0.49703878,
 0.46606138,
 0.75259066,
 0.31260177,
 0.6609633,
 0.31887963,
 0.7741204,
 0.27057046,
 0.66150296,
 0.34389865,
 0.87218475,
 0.14609902,
 0.6541497,
 0.33856723,
 0.5462443,
 0.42175418,
 0.521305,
 0.45328853,
 0.800982,
 0.25545135,
 0.7255122,
 0.26031727,
 0.7967296,
 0.31771147,
 0.6508306,
 0.3487359,
 0.6544872,
 0.31844813,
 0.17745602,
 0.75326276,
 0.8638974,
 0.19800606,

In [26]:
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score
#Returns accuracy,auc,f1_score
def score(y,pred):
    return accuracy_score(y,pred),roc_auc_score(y,pred),f1_score(y,pred)
pred_test=model.predict(test_dataset)
# print(f'The accuracy on the test set is {accuracy_score(df_test_y,pred_test_gb)} and the AUC is {roc_auc_score(df_test_y,pred_test_gb)}')

400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step


c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [36]:
model.evaluate(test_dataset,metrics=['roc_auc_score'])

ValueError: Arguments not recognized: {'metrics': ['roc_auc_score']}

In [21]:
train_df

,label,type,x,y,area,perimeter,neighbours,max neighbour distance,min neighbour distance,max vertices distance,min vertices distance,max vertices-point distance,min vertices-point distance,distance to center,activity,particle type,image_id
0,1,1,8.71810,3.963970,0.992258,3.836169,8.0,1.756655,0.822074,1.436207,0.015668,0.884407,0.509452,0.129601,1.0,1.0,0.0
1,2,1,19.83950,28.755500,1.222737,4.237809,6.0,1.405774,1.081656,1.501896,0.201827,0.794314,0.651656,0.021338,1.0,1.0,0.0
2,3,1,14.11380,0.123613,1.021340,3.896370,6.0,1.549040,0.911955,1.440380,0.250894,0.854049,0.552541,0.084406,1.0,1.0,0.0
3,4,1,27.99440,27.836500,0.996716,3.825848,6.0,1.444316,0.855382,1.407457,0.115618,0.733849,0.540194,0.057512,1.0,1.0,0.0
4,5,1,20.58560,16.870600,0.861796,3.562893,6.0,1.212920,0.901367,1.226116,0.105015,0.700601,0.527187,0.036320,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599995,996,4,17.00900,16.738400,1.398074,4.562085,6.0,1.706074,1.043705,1.564971,0.018557,0.901003,0.571301,0.128033,0.0,0.0,1999.0
1599996,997,4,27.12020,3.015330,0.644134,3.089516,6.0,1.205705,0.792358,1.066991,0.024285,0.604872,0.476485,0.031195,0.0,0.0,1999.0
1599997,998,4,28.79020,6.405500,0.777852,3.426046,6.0,1.183314,0.789633,1.177903,0.128303,0.631295,0.570359,0.036724,0.0,0.0,1999.0
1599998,999,4,11.37550,25.538700,0.648328,3.096628,5.0,0.859703,0.801224,1.098747,0.505184,0.608810,0.491049,0.020731,0.0,0.0,1999.0


In [ ]:
pred_test
probabilities = keras.activations.softmax(tf.convert_to_tensor(pred_test)).numpy()
np.floor(probabilities[:,:,0]*2)
len(probabilities[:][0][:])

400

In [ ]:
np.array(test_df.activity,)

array([1., 1., 1., ..., 0., 0., 0.])

In [77]:
roc_auc_score(np.floor(probabilities[:,:,0]*2),np.array(test_df.activity))


ValueError: Found input variables with inconsistent numbers of samples: [400, 400000]